In [ ]:
#Drive connection
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"

/content
mkdir: cannot create directory ‘gdrive’: File exists
/content/gdrive
mkdir: cannot create directory ‘My Drive’: File exists
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [ ]:
#install the necessary packages
!pip install git+https://github.com/NVIDIA/NeMo.git#egg=nemo_toolkit[all]
!pip install transformers
!pip install wavio
!pip install SpeechRecognition
!pip install jiwer

DEPRECATION: git+https://github.com/NVIDIA/NeMo.git#egg=nemo_toolkit[all] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/NeMo.git to /tmp/pip-install-7dal473_/nemo-toolkit_8b6e84f2b5d1410d8589c04958456ebb
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-7dal473_/nemo-toolkit_8b6e84f2b5d1410d8589c04958456ebb
  Resolved https://github.com/NVIDIA/NeMo.git to commit 72132a200d915e88f461bbcb5db1e2bd54d8ed93
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... canceled
ERROR: Operation cancelled by user
Looking in indexes: https://

In [ ]:

import pandas as pd
import librosa
import wavio as wv
import nemo.collections.asr as nemo_asr
import speech_recognition as sr
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import jiwer
import numpy as np
import wave
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Define the main directory
DIR='/content/gdrive/My Drive/Meta_Asr/Main_Dataset/'
#The GroundTruth of the Raw data
Path_Raw_GroundTruth_Data = "/content/gdrive/My Drive/Meta_Asr/Data_Annotation/Audios_Clusturing/ClusterData_RawData_MetaFeatureClustered.csv"
#The GroundTruth of the PreProcessing
Path_Preprocessing_GroundTruth_Data = "/content/gdrive/My Drive/Meta_Asr/Data_Annotation/Audios_Clusturing/ClusterData_PreProcessed_MetaFeatureClustered.csv"
#Path of Clustered RAW_MetaFeaturesData
RAW_MetaFeaturesData = "/content/gdrive/My Drive/Meta_Asr/Data_Annotation/Audios_Clusturing/RawData_MetaFeatureClustered.csv"
#Path of Clustered preprocessed Meta Features
PreProcessed_MetaFeaturesData = "/content/gdrive/My Drive/Meta_Asr/Data_Annotation/Audios_Clusturing/PreProcessed_MetaFeatureClustered.csv"



#define the candidate pre_trained models
CANDIDATE_PRE_TRAINED_MODELS={
    0:"QuartZnet_Model",
    1:"Wav2Vec50_Model",
    2:"Wav2Vec960_Model",
    3:"speech2Text_Model"
   }

Temp_Audio_File= "Audio_resampling"
QuartZnet_Model = "QuartzNet15x5Base-En"
Wav2Vec_Model = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
Wav2Vec960_Model = "facebook/wav2vec2-base-960h"
Speech2Text = sr.Recognizer()



#config setup
def setup(filename):
  try:
    os.mkdir(filename)
  except OSError:
    pass


#load data
def load_data(path):
  df = pd.read_csv(path)
  return df


#resampling each audio to sample rate 16000
def resampling_audio(input_audio, target_sr=16000):
  temp_new_path="/content/rs_audio.wav"
  audio,sr = librosa.load(input_audio,sr=target_sr)
  wv.write(temp_new_path,audio,target_sr,sampwidth=2)
  return temp_new_path


#Wav2vec prediction
def wav2VecPrediction(resampled_audio):
    #load model
    model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
    tokenizer = Wav2Vec2Tokenizer.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
    #preprocessing signzl
    audio, rate = librosa.load(resampled_audio)
    # to tokens
    input_values = tokenizer(audio, return_tensors = "pt").input_values
    logits = model(input_values).logits
    prediction = torch.argmax(logits, dim = -1)
    # decode results
    transcription = tokenizer.batch_decode(prediction)[0]
    #wer_Wav2Vec = jiwer.wer(transcription,real_text)
    return transcription

def wav2Vec_960_Prediction(resampled_audio):
  #load model
  model = Wav2Vec2ForCTC.from_pretrained(Wav2Vec960_Model)
  tokenizer = Wav2Vec2Tokenizer.from_pretrained(Wav2Vec960_Model)
  #preprocessing signzl
  audio, rate = librosa.load(resampled_audio)
  # to tokens
  input_values = tokenizer(audio, return_tensors = "pt").input_values
  logits = model(input_values).logits
  prediction = torch.argmax(logits, dim = -1)
  # decode results
  transcription = tokenizer.batch_decode(prediction)[0]
  return transcription

#QuartZnet prediction
def quartZnetPrediction(resampled_audio):
  # Load Nemo ASR pre-trained model
  model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name=QuartZnet_Model)
  files = [resampled_audio]
  for fname, transcription in zip(files, model.transcribe(paths2audio_files=files)):
    raw_text = transcription
  return raw_text

#speech2Text prediction
def speech2textPrediction(resampled_audio):
  audio, sr_lib = librosa.load(resampled_audio)
  with sr.AudioFile(resampled_audio) as source:
    audio =  Speech2Text.record(source)
    text =  Speech2Text.recognize_google(audio)
  return text


#model evaluation using WER
def evaluate_model(prediction_text,real_text):
  wer_res = jiwer.wer(prediction_text,real_text)
  return wer_res



#select the candidate pre_trained model
def model_selection(df):
  list_models=[]
  #load data
  for i in range(len(df)):
   input_audio = df['Reference_Audios'][i]

   real_text = df['GroundTruth_Text'][i]
   #resampling audio
   temp_path=resampling_audio(input_audio)
   # wav2Vec Prediction
   wav2vec_res = wav2VecPrediction(temp_path)
   #quartZnet Prediction
   quartZnetPrediction_res = quartZnetPrediction(temp_path)
   #prediction wav2vec 690 model
   wa2vec_690_res = wav2Vec_960_Prediction(temp_path)
   #Speech2text prediction
   speech2textPrediction_res = speech2textPrediction(temp_path)
   #model evaluation  prediction
   wer_Wav2Vec50 = evaluate_model(wav2vec_res,real_text)
   wer_quartZnet = evaluate_model(quartZnetPrediction_res,real_text)
   wer_speech2text = evaluate_model(speech2textPrediction_res,real_text)
   wer_wav2vec690 =  evaluate_model(wa2vec_690_res,real_text)
   #extract the min wer
   list_wer_res = [wer_Wav2Vec50,wer_quartZnet,wer_speech2text,wer_wav2vec690]
   index_min_wer = list_wer_res.index(min(list_wer_res))
   #select the best pretrained model
   for i in range(len(list_wer_res)):
     if index_min_wer == i:
       print('the best model is '+str(CANDIDATE_PRE_TRAINED_MODELS[i]))
       list_models.append(CANDIDATE_PRE_TRAINED_MODELS[i])
   #add the pretrained model columns
  df["selection_model"] = list_models
  return  df


def labelingAudioData(groundTruthDataPath):
  #load data
  df = load_data(groundTruthDataPath)
  #Model Selection
  df1= model_selection(df)
  #save the data in csv file
  df1.to_csv(groundTruthDataPath,index=0)
  return  df1



def metaFeaturesData(DIR,groundTruthData, MetaFeaturesData, data_name):
  # Load data
  df1 = labelingAudioData(groundTruthData)
  df2 = pd.read_csv(MetaFeaturesData)
  df2['selection_model'] = 0
  # Merge the dataframes on the 'cluster' column
  df_merged1 = pd.merge(df2, df1[['cluster', 'selection_model']], on='cluster', how='left')
  df_merged1.drop('selection_model_x', axis=1, inplace=True)
  df_merged1.rename(columns={'selection_model_new': 'selection_model'}, inplace=True)
  # Save the dataframe to a csv file
  df_merged1.to_csv(DIR+data_name + '.csv', index=False)


















In [ ]:
if __name__ == "__main__":
  #create the dataset files
  setup(DIR)
  #labled the main metaFeatures for raw data
  data_name='Main_RawMetaData'
  metaFeaturesData(DIR,Path_Raw_GroundTruth_Data,RAW_MetaFeaturesData,data_name)
  ##labled the main metaFeatures  data
  data_name='Main_PreProcessedMetaData'
  metaFeaturesData(DIR,Path_Preprocessing_GroundTruth_Data,PreProcessed_MetaFeaturesData,data_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


[NeMo I 2023-06-16 13:18:36 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.19.0rc0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2023-06-16 13:18:36 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.19.0rc0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2023-06-16 13:18:36 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-06-16 13:18:37 features:289] PADDING: 16
[NeMo I 2023-06-16 13:18:37 save_restore_connector:249] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.19.0rc0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


the best model is Wav2Vec50_Model


...

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


[NeMo I 2023-06-16 13:29:09 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.19.0rc0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2023-06-16 13:29:09 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.19.0rc0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2023-06-16 13:29:09 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-06-16 13:29:09 features:289] PADDING: 16
[NeMo I 2023-06-16 13:29:10 save_restore_connector:249] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.19.0rc0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


the best model is Wav2Vec960_Model
